In [1]:
import pandas as pd

In [3]:
df = pd.read_excel('../data/chatgpt_explain_data/annotated-June_exp1_explain_and_expand_NO.1.xlsx', sheet_name='Sheet1')
df

,Unnamed: 0,line_id,log_message,template,responses,level,mancraft,weight,target,label_info,full_log
0,0.0,1,ras kernel info instruction cache parity error...,instruction cache parity error corrected,This log is indicating that a parity error has...,5,False,2.230936e-02,0,-,1117838570 2005.06.03 r02-m1-n0-c:j12-u11 2005...
1,1.0,1,ras kernel info instruction cache parity error...,instruction cache parity error corrected,"This log indicates that the RAS (Reliability, ...",1,False,2.230936e-02,0,-,1117838570 2005.06.03 r02-m1-n0-c:j12-u11 2005...
2,2.0,1,ras kernel info instruction cache parity error...,instruction cache parity error corrected,This log indicates that a parity error was det...,3,False,2.230936e-02,0,-,1117838570 2005.06.03 r02-m1-n0-c:j12-u11 2005...
3,3.0,1,ras kernel info instruction cache parity error...,instruction cache parity error corrected,This log indicates that a parity error was det...,2,False,2.230936e-02,0,-,1117838570 2005.06.03 r02-m1-n0-c:j12-u11 2005...
4,4.0,1,ras kernel info instruction cache parity error...,instruction cache parity error corrected,This log indicates that there was a parity err...,4,False,2.230936e-02,0,-,1117838570 2005.06.03 r02-m1-n0-c:j12-u11 2005...
...,...,...,...,...,...,...,...,...,...,...,...
1513,1505.0,4747244,ras kernel info 1 torus non-crc error(s) (dcr ...,(.*) torus non-crc errors dcr (.*) detected an...,This log entry is indicating that there was an...,3,False,2.106166e-07,0,-,1136311192 2006.01.03 r55-m0-n9-c:j06-u11 2006...
1514,1506.0,4747244,ras kernel info 1 torus non-crc error(s) (dcr ...,(.*) torus non-crc errors dcr (.*) detected an...,This log message indicates that there was an e...,4,False,2.106166e-07,0,-,1136311192 2006.01.03 r55-m0-n9-c:j06-u11 2006...
1515,1507.0,4747244,ras kernel info 1 torus non-crc error(s) (dcr ...,(.*) torus non-crc errors dcr (.*) detected an...,This log message indicates that there was an e...,4,False,2.106166e-07,0,-,1136311192 2006.01.03 r55-m0-n9-c:j06-u11 2006...
1516,1508.0,4747244,ras kernel info 1 torus non-crc error(s) (dcr ...,(.*) torus non-crc errors dcr (.*) detected an...,This log message is indicating that there was ...,1,False,2.106166e-07,0,-,1136311192 2006.01.03 r55-m0-n9-c:j06-u11 2006...


## supervised finetune data

In [4]:
# find log message and the response with the smallest level in every group of line_id (best response)
sft_df = df.groupby('line_id').apply(lambda x: x.loc[x['level'].idxmin(), ['log_message', 'responses', 'level']])

In [5]:
sft_df.reset_index(inplace=True)

In [6]:
sft_df.rename(columns={'responses': 'response'}, inplace=True)
sft_df

,line_id,log_message,response,level
0,1,ras kernel info instruction cache parity error...,"This log indicates that the RAS (Reliability, ...",1
1,1477,ras linkcard info midplaneswitchcontroller per...,"This log is related to the RAS (Reliability, A...",1
2,2735,ras kernel info generating core.304,This log entry is indicating that the RAS (Rel...,1
3,4882,ras kernel info 1 ddr errors(s) detected and c...,This log message indicates that there was an e...,1
4,4883,ras kernel info 3450051 l3 edram error(s) (dcr...,EDRAM (Embedded Dynamic Random Access Memory) ...,1
...,...,...,...,...
297,4673522,ras app fatal ciod: login chdir(/p/gb1/stella/...,This log message indicates that a RAS (Remote ...,1
298,4693410,ras kernel fatal error receiving packet on tre...,"This log indicates that the RAS (Reliability, ...",1
299,4702885,ras kernel fatal rts assertion failed: `vaddr ...,This log message is indicating a fatal error t...,1
300,4707636,"ras kernel info ciod: for node 29, incomplete ...",This log message is indicating an issue relate...,1


In [7]:
# transform the enter key to \n
sft_df['response'] = sft_df['response'].str.replace('\n', r'\n')

In [16]:
train_df = pd.read_csv('../data/bgl/train80_bgl_structured_unique.csv')

In [17]:
train_line_id = train_df['line_id'].values
train_line_id

array([      1,    1477,    2735,    4882,    4883,    4884,    4885,
          4903,    4904,    4988,    5065,   14297,   14729,   14730,
         14731,   14736,   14738,   17742,   29727,   33892,   33895,
         33896,   33897,   33899,   33900,   33901,   33903,   33904,
         33905,   33906,   33907,   33908,   33909,   33911,   33912,
         33913,   33915,   33916,   33917,   33918,   33919,   33920,
         33921,   33923,   33924,   33926,   33927,   33935,   33937,
         33950,   34592,   34898,  123236,  123238,  142853,  142854,
        184955,  190328,  190329,  190333,  190334,  190335,  190336,
        190337,  190338,  190339,  190341,  190342,  190343,  190344,
        190346,  190352,  190354,  190356,  190362,  190365,  190368,
        190369,  190370,  190373,  190374,  190375,  190376,  190377,
        190387,  190389,  190390,  190392,  190393,  190394,  190396,
        190398,  190399,  190400,  190401,  190404,  191408,  191409,
        191411,  191

In [18]:
sft_train_df = sft_df[sft_df['line_id'].isin(train_line_id)]
sft_train_df

,line_id,log_message,response,level
0,1,ras kernel info instruction cache parity error...,"This log indicates that the RAS (Reliability, ...",1
1,1477,ras linkcard info midplaneswitchcontroller per...,"This log is related to the RAS (Reliability, A...",1
2,2735,ras kernel info generating core.304,This log entry is indicating that the RAS (Rel...,1
3,4882,ras kernel info 1 ddr errors(s) detected and c...,This log message indicates that there was an e...,1
4,4883,ras kernel info 3450051 l3 edram error(s) (dcr...,EDRAM (Embedded Dynamic Random Access Memory) ...,1
...,...,...,...,...
230,3625952,ras kernel fatal ddrsize == 256*1024*1024 || d...,This log message indicates a fatal error in th...,2
231,3625953,ras kernel fatal rts assertion failed: `person...,This log is indicating a fatal error that occu...,2
232,3645731,ras mmcs fatal l3 major internal error,This log entry indicates a fatal error in the ...,2
233,3662684,ras kernel fatal error receiving packet on tre...,This log message indicates that there was a fa...,1


In [19]:
sft_df.to_csv('../data/rlhf/annotated-best-June_exp1_explain_and_expand_NO.1.csv', index=False)
sft_train_df.to_json('../data/rlhf/train80_chatgpt_sft.json', orient='records', lines=False)

## reward model data

In [10]:
df.dropna(subset=['level'], inplace=True)

In [11]:
RM_data = []

In [12]:
def get_all_composition_of_sub_df(sub_df:pd.DataFrame):
    sub_df = sub_df.sort_values(by='level', ascending=True).reset_index(drop=True)
    for i in range(len(sub_df)-1):
        for j in range(i+1, len(sub_df)):
            RM_data.append({
                'input_log': sub_df.loc[i, 'log_message'],
                'response_a': sub_df.loc[i, 'responses'],
                'response_b': sub_df.loc[j, 'responses']
            })

In [13]:
df.groupby('line_id').apply(get_all_composition_of_sub_df)

""


In [15]:
# save RM_data to json file
import json
with open('../data/rlhf/log_supplement_comparison_data.json', 'w') as f:
    json.dump(RM_data, f, indent=4)

## PPO data

In [ ]:
import pandas as pd
train_df = pd.read_csv('/data/user/nyf/LAB/LogTurbo/data/bgl/train80_bgl_structured_fillall.csv')

In [ ]:
train_df.log_message = train_df.log_message.str.lower()
train_df

In [ ]:
unique_train_df = train_df.drop_duplicates(subset=['log_message'], keep='first')
unique_train_df

In [ ]:
unique_train_df.sample(n=600, random_state=1006).to_csv('../data/rlhf/train_600', index=False)